<a href="https://colab.research.google.com/github/RL-Starterpack/rl-starterpack/blob/main/exercises/PG.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# RL Tutorial - **Policy Gradient Exercise**

## Setup

In [ ]:
#@title Run this cell to clone the RL tutorial repository and install it
try:
  import rl_starterpack
  print('RL-Starterpack repo succesfully installed!')
except ImportError:
  print('Cloning RL-Starterpack package...')

  !git clone https://github.com/RL-Starterpack/rl-starterpack.git
  print('Installing RL-StarterPack package...')
  !pip install -e rl-starterpack[full] &> /dev/null
  print('\n\n~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')
  print('Please restart the runtime to use the newly installed package!')
  print('Runtime > Restart Runtime')
  print('~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~')

In [ ]:
#@title Run this cell to install additional dependencies (will take ~30s)
!apt-get remove ffmpeg > /dev/null # Removing due to restrictive license
!apt-get install -y xvfb x11-utils > /dev/null

In [ ]:
#@title Run this cell to import the required libraries
try:
  from rl_starterpack import OpenAIGym, PG, experiment, vis_utils
except ImportError:
  print('Please run the first cell! If you already ran it, make sure to restart the runtime after the package is installed.\n')
  raise
import pandas as pd
import numpy as np
import torch
import gym
import torchviz
from itertools import chain
%matplotlib inline
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

# Setup display to show video renderings
if 'display' not in globals():
    display = Display(visible=0, size=(1400, 900))
    display.start()

## Exercise

### My Policy Gradient Agent
As a first step, we will be implementing two key parts of the Policy Gradient agent:

1. The `reward_to_go` function to be used in the REINFORCE algorithm.
1. The `surrogate_loss` function to be used for gradient descent.

Recall from our REINFORCE algorithm:


>For each timestep in our trajectory $(t= 1, ..., T)$:
  1. Compute the `reward_to_go`: $\sum_{k=t}^{T}\gamma^{k-t}R(k)$
  1. Update the policy parameters using: $\theta\gets\theta+\alpha R(t)\nabla_\theta \log(\pi_\theta(a_t | s_t))$

And that we minimize a `surrogate_loss` to give us the desired gradient above:

> $L(s,a) = -R(\tau)\log(\pi_\theta(a | s))$

In practice we use the mean across a whole rollout:

> $Loss = \frac{-\sum_{t=1}^{T}R(t)\log(\pi_\theta(a_t | s_t)}{T}$

Where here $R(t)$ is the `reward_to_go` at timestep $t$.

If you need a refresher, you can refer back to the slides on the [tutorial home page](http://rl-starterpack.github.io/). If you're curious about the full implementation, it can be found in the [repo for this tutorial](https://github.com/RL-Starterpack/rl-starterpack/blob/main/rl_starterpack/agents/pg.py).

In [ ]:
# TODO: Fill in these two methods
class MyPolicyGradient(PG):
  @staticmethod
  def reward_to_go(reward, terminal, discount):
      """
      Reward to go implementation.
      Args:
          reward (np.ndarray[float]): 1-D array of rewards for the current, complete episode rollout. For example:
              [0., 0., 0., 0., 1.] would represent a rollout with 5 timesteps where the agent received a reward of 1
              at the last timestep and a reward of 0 at every other timetstep.
          terminal (np.ndarray[int]): 1-D array representing whether or not the timestep is "terminal" (i.e. the 
              episode has finished). Same dimension as `reward`. For example:
              [0., 0., 0., 0., 1.] would represent a rollout with 5 timesteps where the last timestep is terminal.
          discount (float): Discount factor in the range [0,1].
      Returns:
          np.ndarray: 1-D array representing the reward to go at every timestep. Should have the same dimensions as 
              `reward` and `terminal`.
      """
      raise NotImplementedError

  @staticmethod
  def surrogate_loss(rollout_value, log_prob):
      """
      Surrogate loss implementation.
      Args:
          rollout_value (torch.Tensor[float]): 1-D array of reward_to_go, as computed by reward_to_go.
          log_prob (torch.Tensor[float]): 1-D array of log probabilities for the agent actions given the current state.
              Same dimension as rollout value.
      Returns:
          torch.Tensor[float]: Scalar of the current loss. Note you may need to use the `my_tensor.mean()` function.
      """
      raise NotImplementedError

Run the cells below to test that your implementation works!

In [ ]:
def test_reward_to_go(pg):
    reward = np.array([0, 0.2, 0.5, 0.8])
    terminal = np.array([0, 0, 0, 1])
    discount = 0.9
    r2g = pg.reward_to_go(reward, terminal, discount)
    expected_r2g = np.array([1.1682, 1.298 , 1.22  , 0.8])

    assert isinstance(r2g, np.ndarray)
    np.testing.assert_allclose(r2g, expected_r2g) 

def test_surrogate_loss(pg):
    rollout_value = torch.tensor([0.5, 1.5, 2.0, 3.5])
    log_prob = torch.tensor([-0.5, -0.6, -0.7, -0.8])
    loss = pg.surrogate_loss(rollout_value, log_prob)
    expected_loss = torch.tensor(1.3375)

    assert isinstance(loss, torch.Tensor)
    assert torch.allclose(loss, expected_loss) 

In [ ]:
tests_pass = False
try:
    test_reward_to_go(MyPolicyGradient)
    test_surrogate_loss(MyPolicyGradient)
    tests_pass = True
except NotImplementedError:
    print('Your implementation is incomplete!\n')

In [ ]:
#@title _<sub><sup>SOLUTION: Expand this cell to see a solution for the MyPolicyGradient implementation </sup></sub>_
class MyPolicyGradientSolution(PG):
    @staticmethod
    def reward_to_go(reward, terminal, discount):
        num_timesteps = reward.shape[0]
        rollout_value = reward
        for n in range(num_timesteps - 2, -1, -1):
            if terminal[n] == 0:
                rollout_value[n] += discount * rollout_value[n + 1]
        return rollout_value

    @staticmethod
    def surrogate_loss(rollout_value, log_prob):
        # Surrogate loss:  -(R[t:] * log pi(s_t, a_t))
        return -(rollout_value * log_prob).mean()

test_reward_to_go(MyPolicyGradientSolution)
test_surrogate_loss(MyPolicyGradientSolution)

In [ ]:
# If the tests pass, then we use your implementation here on out, otherwise use repo implementation
if tests_pass:
    PolicyGradient = MyPolicyGradient
else:
    PolicyGradient = PG

### CartPole... Again! 
Now let's test that our agent works on cartpole.

In [ ]:
env = OpenAIGym('CartPole', max_timesteps=300)  # We set this to 300 to prevent training from taking too long, but feel free to increase this 

We model our policy with a neural network which takes in the current state as the input and outputs a distribution over actions.

In [ ]:
state_obs_dim = env.state_space['shape'][0]  # Size of vector representing the state observations
num_actions = env.action_space['num_values']
hidden_size = 16 

network_fn = (lambda: torch.nn.Sequential(
    torch.nn.Linear(in_features=state_obs_dim, out_features=hidden_size),
    torch.nn.Tanh(),
    torch.nn.Linear(in_features=hidden_size, out_features=num_actions)
))

We can inspect this input-output relationship by calling our network. The output corresponds to the logits (unnormalized predictions) of the actions: apply force left or right.

In [ ]:
example_policy_network = network_fn()
zero_state = np.zeros(shape=env.state_space['shape'], dtype=np.float32)
example_policy_network(torch.tensor(zero_state))

Now let's build our agent with this policy network.

In [ ]:
agent = PolicyGradient(
    state_space=env.state_space, action_space=env.action_space,
    network_fn=network_fn, learning_rate=1e-3,
    discount=0.99, 
    normalize_returns=True
)

Note: You may notice that above we have `normalize_returns=True`, which subtracts the mean and divides by the standard deviation of the returns within the current episode. In practice, normalizing returns helps ensure stability by controlling the variance of the policy gradient estimator. More information can be found on [Andrej Karpathy's blog](http://karpathy.github.io/2016/05/31/rl/) under "More general advantage functions". Feel free to try the agent without the normalization and see how it performs!

Reward normalization can also be seen as a very simple form of advantage actor critic (A2C), since it similarly normalizes the variance and centers the returns around 0. We will see A2C in the next tutorial section, where we discuss actor-critic (AC) methods.

In [ ]:
train_returns = experiment.train(agent, env, num_episodes=1000)
eval_returns = experiment.evaluate(agent, env, num_episodes=500)

In [ ]:
vis_utils.draw_returns_chart(train_returns, smoothing_window=20)

Let's visualise our performance! If your agent managed to balance the pole for >300 timesteps, try increasing max_timesteps below to see if it balance indefinitely. Also note that since the cartpole has a random starting state, it might not solve it everytime, so try running the visualisation a few times to see different episodes.

_Note: to show videos in Colab, we need to render them as gifs._

In [ ]:
# To show longer balancing run this before creating the gif: env = OpenAIGym('CartPole', max_timesteps=500)
vis_utils.show_episode_as_gif(ipythondisplay, agent, env)

As you may have noticed, Policy Gradient tends to converge faster (and more consistently) than DQN.

### Pendulum

Recall that one of the advantages of Policy Gradient is that we can work with continuous action spaces. So let's try our implementation on an environment with a continuous action space: [Pendulum](https://gym.openai.com/envs/Pendulum-v0/)

> The inverted pendulum swingup problem is a classic problem in the control literature. In this version of the problem, the pendulum starts in a random position, and the goal is to swing it up so it stays upright.

In [ ]:
env = OpenAIGym('Pendulum', max_timesteps=100)

In [ ]:
print('Action Information\n')
pd.DataFrame.from_dict(env.action_space)

In [ ]:
print('State Information (theta is pole angle from vertical) \n')
pd.DataFrame.from_dict({'Observation': ['cos(theta)', 'sin(theta)', 'angular velocity'],
                        'min_value': env.state_space['min_value'],
                        'max_value': env.state_space['max_value'], 
                        })

Now we define our PG network and run it on the new environment.

In [ ]:
state_obs_dim = env.state_space['shape'][0]  # Size of vector representing the state observations
action_dim = env.action_space['shape'][0]
hidden_size = 16 

network_fn = (lambda: torch.nn.Sequential(
    torch.nn.Linear(in_features=state_obs_dim, out_features=hidden_size),
    torch.nn.Tanh(),
    torch.nn.Linear(in_features=hidden_size, out_features=action_dim)
))

agent = PolicyGradient(
    state_space=env.state_space, action_space=env.action_space,
    network_fn=network_fn, learning_rate=1e-3,
    discount=0.9, 
    normalize_returns=True
)

In [ ]:
train_returns = experiment.train(agent, env, num_episodes=2000)
eval_returns = experiment.evaluate(agent, env, num_episodes=100)
vis_utils.draw_returns_chart(train_returns, smoothing_window=50)

In [ ]:
vis_utils.show_episode_as_gif(ipythondisplay, agent, env)

Most likely the agent failed to "swing up" the pendulum. Try running the visualisation a few times to see how it performs with different random pole initialisations.


The "natural" environment reward is a negative cost, so the best we can do is 0 reward. The cost is a function of the angle (higher if the pole is further from being upright), angular momentum (higher for higher angular momentum) and the applied torque (higher for more torque).

We can do somewhat better than our attempt above, by using our own reward shaping function. This function rewards the agent for having high angular momentum at the bottom half of the pendulum swing, and making it to the top half of the swing with less momentum.

In [ ]:
def reward_shaping_fn(reward, terminal, state):
    cos_theta, sin_theta, ang_momentum = state
    if 0 <= cos_theta <= 1: # upper half
        return (25 - np.abs(ang_momentum)*0.5)*5, terminal
    return cos_theta + np.abs(ang_momentum)*0.5, terminal  # cos_theta will be negative here

In [ ]:
env = OpenAIGym('Pendulum', max_timesteps=100)
agent = PolicyGradient(
    state_space=env.state_space, action_space=env.action_space,
    network_fn=network_fn, learning_rate=1e-3,
    discount=0.9, 
    normalize_returns=True
)

In [ ]:
train_returns = experiment.train(agent, env, num_episodes=2000, reward_shaping_fn=reward_shaping_fn)
eval_returns = experiment.evaluate(agent, env, num_episodes=100)
vis_utils.draw_returns_chart(train_returns, smoothing_window=50)

In [ ]:
vis_utils.show_episode_as_gif(ipythondisplay, agent, env)

Once again try running the visualisation a few times to see how it performs with different random pole initialisations.

As you've most likely observed, it ends up doing somewhat better but far from solves the environment. It turns out that this is quite a challenging problem, because the agent can't just "react" to the observed pole position. It needs to learn how the torque it applies to the pole will impact its position and angular momentum and plan around that. To solve this, we may need to use some more advanced techniques (a solution using Deep Deterministic Policy Gradient can be found [here](https://github.com/lirnli/OpenAI-gym-solutions/blob/master/Continuous_Deep_Deterministic_Policy_Gradient_Net/DDPG%20Class%20ver2%20(Pendulum-v0).ipynb)).

If youv'e finished early feel free to play around with the hyperparameters, neural network and reward function above to see if you can find a better solution to the Pendulum environment!

## Leaderboard

Once you have completed the exercises above consider submitting your scores to our PG leaderboard using [this form](https://forms.gle/jKaNZKS9vkBaWY9y7).

Note: to compute the "mean evaluation return" you can do `eval_returns.mean()`.